In [31]:
'''
Import des librairies nécéssaires pour faire tourner le code
---------------------
librairies externes
---------------------
si ça ne tourne pas, c'est qu'il faut installer le package en tapant dans la console (Ctrl+ù sur VScode):

conda install package

'''
import pandas as pd
import os
from datetime import datetime
from time import sleep
from openpyxl import Workbook
from openpyxl import load_workbook
import plotly_express as px
import plotly.graph_objects as go
import folium
from folium.plugins import Draw

'''
Libraires internes
---------------------
Si ça ne tourne pas il faut ajouter au début de votre code:

import sys
sys.path.append(r'chemin d'accès au dossier contenant la bibliothèque interne')
---------------------
Exemple:
> from library.PY_lib.coordinates import *

le chemin d'accès du dossier "library" est D:\Programmes\E.nova\python\PyEnova\flight_track\library

il faut donc ajouter au dessus :

import sys
sys.path.append(r'D:\Programmes\E.nova\python\PyEnova\flight_track')

L'opération n'est en principe a faire qu'une seule fois, tant que vos bibliothèques sont rangées au même endroit, 
d'où l'importance de ne pas éparpiller les dossiers de librairies un peu partout
'''

from library.PY_lib.coordinates import *
from library.PY_lib.read_excel import *
from library.PY_lib.calculate_distance import *
from library.PY_lib.user_define import *
from library.quad_filter import *
from library.PY_lib.flight_scrapper import *
from library.PY_lib.format_json import *
from library.PY_lib.json_to_csv import *
from library.PY_lib.json_to_excel import *



# from library.PY_lib.data_retriever import *


In [32]:
'''
Option permettant de choisir entre générer les données ou utiliser des données pré-enregistrées
le dossier en question doit être enregistré au même endroit que le NB
'''
ip=input('rentrez G pour générer des données, ou un nom de dossier pour en extraire les données')

if ip != 'G':
    folder = ip
else : folder='input'


In [33]:
'''
Paramètres de génération de données:
'''
n = 1
'''automatisation (0 pour oui, 1 pour non)'''
r = 1
'''request left'''
t = 3600 
'''temps entre chaque requete (en secondes)'''
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
'''date d'extraction des données'''

'''
Paramètres pour délimiter la zone
'''
choice = 2
'''1=quadrilatère, 2 = circulaire'''
taille_zone = 100
'''en km si choix 2'''


'en km si choix 2'

In [34]:
'''
Fonction pour aller récupérer les données via API
'''
def get_data(n, request_left, attente): #, file_name:str
    file_name = f'{current_time}_flight'
    json_file = f'tempo.json'
    # csv_file = f'tab_{namefile}.csv'
    excel_file = f'{os.path.join("input",file_name)}.xls'
    with open(json_file, 'w') as file:
        pass
    # with open(csv_file, 'w') as file:
    #     pass
    with open(excel_file, 'w') as file:
        pass
    scrap_plane(json_file)
    format_json(json_file, json_file)
    # json_to_csv(json_file, csv_file)
    json_to_excel(json_file, excel_file)
   
    print("all operations done")
    if n == 0 and request_left > 0:
        print(f'waiting {attente} seconds to proceed further, {request_left} request(s) left.')
        request_left = request_left - 1
        sleep(attente)
        get_data(n, request_left, attente)

In [35]:
'''
enregistre les données récupérées via API
'''
if ip =='G':
    get_data(n, r - 1, t) #, file_name
    
    file_name = f'{current_time}_flight'
    excel_file = f'{os.path.join("input",file_name)}.xls'
    df=pd.read_excel(excel_file)
    df['Date']=current_time
    df.to_excel(excel_file, index=False)
    

In [36]:
'''
Fonction qui filtre les données en fonction de la zone à étudier.
Pour changer la taille de la zone, il faut modifier les fichiers round_coordinates.txt ou square_coordinates.txt 
'''
def onefiledata(filename, taille_zone, choice, r_latitudes, r_longitude, s_latitude, s_longitude):
    print(filename)
    data = pd.read_excel(filename)
    data = pd.DataFrame(data)

    if choice == 1:
        t_lat = s_latitude
        t_lng = s_longitude
        # filtered_data = []
        for index, row in data.iterrows():
            lat = row['lat']
            lng = row['lng']
            if sort_elements(t_lat[0], t_lat[1], t_lat[2], t_lat[3], lat, t_lng[0], t_lng[1], t_lng[2], t_lng[3], lng) == 0:
                data.loc[index,'distance'] = True
        filtered_data=data.where(data['distance']==True)

    if choice == 2:
        for index, row in data.iterrows():
            lat = row['lat']
            lng = row['lng']
            for ref_lat, ref_lng in zip(r_latitudes, r_longitude):
                data.loc[index,'distance'] = calculate_distance(lat, lng, float(ref_lat), float(ref_lng))
        filtered_data=data.where(data['distance']<=taille_zone)
    # display(filtered_data.dropna())
    return filtered_data.dropna()

In [37]:
'''
Fonction qui charge les données dans le notebook pour postraitement 
et les sauvegardent dans un excel dans le dossier output
'''
round_path = r'D:\Programmes\E.nova\python\PyEnova\flight_track\round_coordinates.txt'
square_path = r'square_coordinates.txt'

r_latitudes, r_longitude = read_coordinates(round_path)
s_latitude, s_longitude = read_coordinates(square_path)

filename = os.listdir(folder)

df=[]
for i in range(len(filename)):
    path2 = os.path.join(folder,filename[i])
    filtered_data=onefiledata(path2,taille_zone, choice, r_latitudes, r_longitude, s_latitude, s_longitude)
    df.append(filtered_data)

if choice==1:
    new_filename = os.path.join('output',f'{current_time}_square_sorted_.xls')
if choice==2: 
    new_filename = os.path.join('output',f'{current_time}_round_sorted_.xls')

filtered_df = pd.concat([*df]) 

filtered_df.to_excel(new_filename, index=False)

print(f"Tout a été enregistré dans le fichier : {new_filename}")

input\2023-10-06_18-15-49_flight.xls
input\2023-10-06_18-18-51_flight.xls
input\2023-10-20_09-45-52_flight.xls
Tout a été enregistré dans le fichier : output\2023-10-20_16-49-38_round_sorted_.xls


C:\Users\arthu\AppData\Local\Temp\ipykernel_11960\2532571716.py:26: FutureWarning:

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.



In [39]:
'''
On a ici filtered_df qui est notre dataFrame de base avec toutes nos données, les calculs sont faits, il ne reste que le postraitement.

Pour couvrir les différents cas sur la carte, on a besoin de différencier les avions qui n'apparaissent qu'une seule fois de ceux qui apparaissent plusieurs fois, 
car on ne les traite pas de la même manière:

Avions uniques : position sur la carte, flèche orientée selon leur cap
Avions qui ont plusieurs occurences: Pareil sauf que les occurences sont reliées entre elles

J'ai donc créé 2 plus petits dataframes:
Avions uniques : df_d
'''
idx=filtered_df['hex'].drop_duplicates(keep=False).index    
'''on créée une liste d'index (ici la colone hex), et on supprime toutes les lignes qui ont plus d'une occurence'''
df_d=filtered_df.loc[idx]
'''on filtre le dataframe de base avec seulement les index uniques'''
df_d=df_d.set_index(['hex', 'Date']).sort_index()
'''On définit les colones hex et date comme index, pour la lisibilité'''

df_p=filtered_df.drop(index=idx)
'''on supprime les lignes avec seulement les index uniques, 
donc on ne garde que les lignes avec des index en double, triple, etc'''
df_p=df_p.set_index(['hex', 'Date']).sort_index()
'''On définit les colones hex et date comme index, pour la lisibilité,
et c'est là qu'avoir du multiindex prend tout son sens.'''



In [46]:
'''
Affichage des deux dataframes:
'''
display(df_d)
display(df_p)

,hex,reg_number,flag,lat,lng,alt,dir,speed,v_speed,squawk,...,dep_iata,arr_icao,arr_iata,airline_icao,airline_iata,aircraft_icao,updated,status,Date,distance
93,346350,EC-NGL,ES,44.119354,6.437222,8221.0,310.4,731.0,8.5,1000.0,...,NCE,LFRB,BES,VOE,V7,A319,1.696609e+09,en-route,2023-10-06_18-15-49,78.695738
112,347344,EC-NVS,ES,43.900762,8.094243,10957.0,72.0,814.0,0.0,1000.0,...,MAD,LIPZ,VCE,IBE,IB,A20N,1.696609e+09,en-route,2023-10-06_18-15-49,94.723539
121,398495,F-HBEV,FR,44.215781,6.851248,10363.0,323.2,822.0,0.0,1000.0,...,CLY,LFPO,ORY,CCM,XK,A320,1.696609e+09,en-route,2023-10-06_18-15-49,75.256658
1228,3C4DD8,D-ACNX,DE,43.955228,7.934810,9525.0,43.0,857.0,6.2,1000.0,...,MRS,EDDM,MUC,DLH,LH,CRJ9,1.696609e+09,en-route,2023-10-06_18-15-49,86.192282
1536,400FE3,G-EZBT,UK,44.381561,6.671448,11590.0,325.9,785.0,-0.3,4044.0,...,CTA,EGKK,LGW,EZY,U2,A319,1.696609e+09,en-route,2023-10-06_18-15-49,96.581363
13480,34368C,EC-LJX,ES,43.661957,7.212237,0.0,225.0,24.0,0.0,5326.0,...,MAD,LFMN,NCE,IBE,IB,CRJX,1.696609e+09,landed,2023-10-06_18-15-49,19.863361
1134,39BAF8,F-HOXY,SI,43.656841,7.207766,0.0,225.0,34.0,0.0,1000.0,...,NCE,LFJL,ETZ,AEH,NL,E145,1.696609e+09,scheduled,2023-10-06_18-18-51,19.228053
2113,471FAC,HA-LZW,HU,44.134687,7.597700,10668.0,244.4,817.0,0.0,4012.0,...,BUD,LPPT,LIS,WZZ,W6,A21N,1.696609e+09,en-route,2023-10-06_18-18-51,79.833191
20,342581,EC-KCU,ES,43.824463,7.712991,10972.0,43.0,922.0,0.3,1000.0,...,BCN,EDDM,MUC,VLG,VY,A320,1.697788e+09,en-route,2023-10-20_09-45-52,63.531564
178,407794,G-UZLB,UK,43.812778,7.249276,11277.0,126.0,778.0,0.3,3370.0,...,BRS,DTNH,NBE,EZY,U2,A20N,1.697788e+09,en-route,2023-10-20_09-45-52,34.550178


reg_number flag        lat       lng      alt  \
hex    Date                                                                
06A1CC 2023-10-06_18-15-49     A7-CGG   QA  43.660904  7.213355      0.0   
       2023-10-06_18-18-51     A7-CGG   QA  43.653259  7.203007      0.0   
39DD44 2023-10-06_18-15-49     F-HXKE   FR  43.204239  6.922409   4572.0   
       2023-10-06_18-18-51     F-HXKE   FR  43.165656  7.226323   4572.0   
3C6669 2023-10-06_18-15-49     D-AISI   DE  43.799927  6.090641  10363.0   
       2023-10-06_18-18-51     D-AISI   DE  44.137253  6.323782  10363.0   
3C674A 2023-10-06_18-15-49     D-AIZJ   DE  43.502884  7.159828    822.0   
       2023-10-06_18-18-51     D-AIZJ   DE  43.631364  7.176165    243.0   
400DB4 2023-10-06_18-15-49     G-EZAS   UK  43.721375  7.020711   5814.0   
       2023-10-06_18-18-51     G-EZAS   UK  43.982834  6.793801   7246.0   
400E4A 2023-10-06_18-15-49     G-GDFJ   UK  43.462234  7.244206   3040.0   
       2023-10-06_18-18-51     G-GDFJ   UK  43.578667  6.964482   4396.0   
440141 2023-10-06_18-15-49     OE-IJX   AT  43.614761  7.311018   1424.0   
       2023-10-06_18-18-51     OE-IJX   AT  43.485535  7.173242   3977.0   
46B821 2023-10-06_18-15-49     SX-NAA   GR  43.329892  7.861829   2575.0   
       2023-10-06_18-18-51     SX-NAA   GR  43.352829  7.546585   1303.0   
4D2104 2023-10-06_18-15-49     9H-BOO   MT  43.447357  6.508633   3268.0   
       2023-10-06_18-18-51     9H-BOO   MT  43.402725  6.822318   2125.0   
4D22B1 2023-10-06_18-15-49     9H-LOZ   IE  43.358289  6.893594  10972.0   
       2023-10-06_18-18-51     9H-LOZ   IE  43.581253  7.306675  10972.0   

                              dir  speed  v_speed  squawk flight_number  ...  \
hex    Date                                                              ...   
06A1CC 2023-10-06_18-15-49  225.0   13.0      0.0  1000.0           511  ...   
       2023-10-06_18-18-51  143.0   13.0      0.0  1000.0           511  ...   
39DD44 2023-10-06_18-15-49   99.0  507.0      0.0  1000.0          41QA  ...   
       2023-10-06_18-18-51  100.0  509.0      0.0  1000.0          41QA  ...   
3C6669 2023-10-06_18-15-49   26.0  853.0      0.3  1057.0           12N  ...   
       2023-10-06_18-18-51   26.0  850.0      0.0  1057.0           12N  ...   
3C674A 2023-10-06_18-15-49  356.4  358.0      0.0  1000.0           55J  ...   
       2023-10-06_18-18-51   45.0  244.0     -3.3  1000.0           55J  ...   
400DB4 2023-10-06_18-15-49  328.1  627.0     13.3  6753.0          48WT  ...   
       2023-10-06_18-18-51  327.8  746.0      8.8  6753.0          48WT  ...   
400E4A 2023-10-06_18-15-49  275.9  452.0      8.5  4070.0          62VC  ...   
       2023-10-06_18-18-51  322.2  625.0     11.1  4070.0          62VC  ...   
440141 2023-10-06_18-15-49  168.1  414.0      5.5  1000.0          908G  ...   
       2023-10-06_18-18-51  279.6  512.0      7.5  1000.0          908G  ...   
46B821 2023-10-06_18-15-49  276.0  533.0     -7.5  2046.0           690  ...   
       2023-10-06_18-18-51  276.0  480.0     -3.3  2046.0           690  ...   
4D2104 2023-10-06_18-15-49  102.0  555.0     -8.1  3322.0          629P  ...   
       2023-10-06_18-18-51  101.1  488.0     -5.2  3322.0          629P  ...   
4D22B1 2023-10-06_18-15-49   53.0  844.0      0.0  5542.0          33ZT  ...   
       2023-10-06_18-18-51   53.0  840.0     -0.4  5542.0          33ZT  ...   

                           dep_icao dep_iata arr_icao arr_iata airline_icao  \
hex    Date                                                                   
06A1CC 2023-10-06_18-15-49     LFMN      NCE     LFBD      BOD          QQE   
       2023-10-06_18-18-51     LFMN      NCE     LFBD      BOD          QQE   
39DD44 2023-10-06_18-15-49     LFML      MRS     LFKC      CLY          CCM   
       2023-10-06_18-18-51     LFML      MRS     LFKC      CLY          CCM   
3C6669 2023-10-06_18-15-49     LEBL      BCN     EDDF      FRA          DLH   
       2023-10-06_18-18-51     LEBL     

In [50]:
'''
Carte avec les avions affichés, après pas mal d'essais sur plotly, je suis finalement passé sur folium
J'ai aussi reçu un coup de main de chatGPT, dont j'ai tout de même du adapter le code pour que ça marche, 
j'ai laissé ses commentaires.
'''
# Create a Folium Map
m = folium.Map(location=[df_p['lat'].mean(), df_p['lng'].mean()], zoom_start=6)

# Iterate through your data and create lines and markers
for i in df_p.index.get_level_values(0):
    locations = []  # Accumulate coordinates for each i    
    for j in df_p.index.get_level_values(1):
        angle = df_p.loc[(i,j), 'dir']  # Angle in degrees
        # Create a marker
        marker = folium.Marker(
            location=(df_p.loc[(i,j), 'lat'], df_p.loc[(i,j), 'lng']),
            icon=folium.DivIcon(
                html=f'<div><div style="transform: rotate({angle}deg); border: 8px solid transparent; border-bottom-color: blue; width: 0; height: 0;"></div></div>'
            ),
            tooltip=i
        )

        m.add_child(marker)

        # Accumulate coordinates for the line
        locations.append((df_p.loc[(i,j), 'lat'], df_p.loc[(i,j), 'lng']))
    
    # Create a line (PolyLine)
    line = folium.PolyLine(
        locations=locations,
        color='blue',
        weight=2,
        opacity=1,
        tooltip=i
    )
    
    m.add_child(line)

for i in df_d.index.get_level_values(0):
    angle = df_d.loc[(i), 'dir']  # Angle in degrees
    # Create a marker
    marker = folium.Marker(
        location=(df_d.loc[(i), 'lat'], df_d.loc[(i), 'lng']),
        icon=folium.DivIcon(
            html=f'<div><div style="transform: rotate({angle}deg); border: 8px solid transparent; border-bottom-color: blue; width: 0; height: 0;"></div></div>'
        ),
        tooltip=i
    )

    m.add_child(marker)


# Add the drawing tools for interactive features (optional)
draw = Draw(export=True)
draw.add_to(m)

# Display the map
display(m)

In [43]:
'''
Tentative 2 avec plotly que je laisse pour la beauté de la chose, mais j'arrivais pas à afficher une flèche orientée selon le cap
'''
fig = go.Figure()

for i in df_p.index.get_level_values(0):
    fig.add_trace(go.Scattermapbox(
        mode="lines+markers", #
        lon=df_p.loc[i, 'lng'],
        lat=df_p.loc[i, 'lat'],
        name=i, 
        marker=dict(
            symbol="arrow",
            size=10,
            angle=df_p.loc[i, 'dir'],
            ),
        ))
    


fig.update_layout(
    margin={'l': 113, 't': 24, 'b': 22, 'r': 115},
    mapbox=dict(style='carto-positron',
                center=go.layout.mapbox.Center(lon=7, lat=43.5),
                pitch=0,
                zoom=6)
)

fig.show()

In [44]:
'''
tentative 1 avec plotly
rapide a coder, mais peu d'options disponibles
'''
fig=px.scatter_geo(
    data_frame=filtered_df,
    lat='lat',
    lon='lng',
    hover_name='reg_number',
    color='flag',
    hover_data=['alt', 'speed'],
    )
fig.update_layout(width=1500, height=900)
fig.write_html(os.path.join('output',f'{current_time}_Flight_map'))
fig


In [52]:
'''
Exemple de graph très rapidement sortable avec plotly
On peut cliquer sur la légende à droite pour afficher/cacher des objets
'''
fig = px.scatter(
    data_frame=filtered_df,
    y='alt',
    x='speed',
    color='Date'
)
fig.show()